# Takes FMS GenScript CSV export and converts it to a proper JSON file for a MongoDB import. 
### Notes 
- This notebook is designed to **Restart & Run all cells** for production 
- FMS CSV export will not have header row. In File Maker export with format type Merge and then update the ext to CSV.
- I used [Studio 3T](https://robomongo.org/) from robomongo to connect to the MongoDB Atlas for the import.

In [1]:
from datetime import datetime
import json 

import pandas as pd

from genscript import pathing  # expands path and protects it

# Read CSV with Byte error

In [5]:
df = pd.read_csv(pathing('../data/GenScript.csv'), encoding = "ISO-8859-1")  # some bytes carried over - hopefully mongodb json import clears it completely

# Fix & Transform Data row to JSON

In [3]:
# Float fix
df['extinction_coefficient'] = df['extinction_coefficient'].apply(lambda x: float(str(x).replace(',', '')))  # could have values as string with comma!
df['molecular_weight'] = df['molecular_weight'].apply(lambda x: float(str(x).replace(',', '')))  # could have values as string with comma!

# Explode aggregated string to list where commas exist
df['backup_dna_location'] = df['backup_dna_location'].apply(lambda x: [_x.strip() for _x in x.split(',')] if isinstance(x, str) else x)  # stored as string with commas!
df['dna_location'] = df['dna_location'].apply(lambda x: [_x.strip() for _x in x.split(',')] if isinstance(x, str) else x)  # stored as string with commas!

# Date to proper ISO 8601
df['date_ordered'] = df['date_ordered'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y').isoformat() if isinstance(x, str) else x)
df['timestamp'] = df['timestamp'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y').isoformat() if isinstance(x, str) else x)

df.head()

,backup_dna_location,clone_name,cloned_sequence,date_ordered,dna_location,extinction_coefficient,full_sequence_aa,full_sequence_nt,genscript_order_item,insert_sequence,leader_sequence_aa,leader_sequence_nt,mature_sequence_nt,molecular_weight,peptide,timestamp,vector_name
0,"[1H-2a, 1H-6a]",BM48-31_RBD_mC2_pCWSec,GTCGACATTGATTATTGACTAGTTATTAATAGTAATCAATTACGGG...,2021-01-26T00:00:00,[14E-3A],30870.0,MGILPSPGMPALLSLVSLLSVLLMGCVAETGNITQLCPFNEVFNIT...,ATGGGGATCCTTCCCAGCCCTGGGATGCCTGCGCTGCTCTCCCTCG...,U0243GA250-2,ACCGGTAACATCACCCAGCTGTGCCCATTCAACGAGGTGTTTAATA...,MGILPSPGMPALLSLVSLLSVLLMGCVA,ATGGGGATCCTTCCCAGCCCTGGGATGCCTGCGCTGCTCTCCCTCG...,GAAACCGGTAACATCACCCAGCTGTGCCCATTCAACGAGGTGTTTA...,22869.3431,ETGNITQLCPFNEVFNITSFPSVYAWERMRITNCVADYSVLYNSSA...,NaN,pCWSec
1,"[1I-7g, 1I-9h]",BtKY72_RBD_mC2_pCWSec,GTCGACATTGATTATTGACTAGTTATTAATAGTAATCAATTACGGG...,2021-01-26T00:00:00,[14E-4A],35340.0,MGILPSPGMPALLSLVSLLSVLLMGCVAETGNITNLCPFGQVFNAS...,ATGGGGATCCTTCCCAGCCCTGGGATGCCTGCGCTGCTCTCCCTCG...,U0243GA250-5,ACCGGTAACATCACCAATCTGTGCCCATTCGGCCAGGTGTTTAACG...,MGILPSPGMPALLSLVSLLSVLLMGCVA,ATGGGGATCCTTCCCAGCCCTGGGATGCCTGCGCTGCTCTCCCTCG...,GAAACCGGTAACATCACCAATCTGTGCCCATTCGGCCAGGTGTTTA...,23084.5661,ETGNITNLCPFGQVFNASNFPSVYAWERLRISDCVADYAVLYNSSS...,NaN,pCWSec
2,"[1I-8g, 1H-3a]",WIV1_RBD_mC2_pCWSec,GTCGACATTGATTATTGACTAGTTATTAATAGTAATCAATTACGGG...,2021-01-26T00:00:00,[14E-5A],39350.0,MGILPSPGMPALLSLVSLLSVLLMGCVAETGNITNLCPFGEVFNAT...,ATGGGGATCCTTCCCAGCCCTGGGATGCCTGCGCTGCTCTCCCTCG...,U0243GA250-8,ACCGGTAACATCACCAATCTGTGCCCCTTCGGCGAGGTGTTTAACG...,MGILPSPGMPALLSLVSLLSVLLMGCVA,ATGGGGATCCTTCCCAGCCCTGGGATGCCTGCGCTGCTCTCCCTCG...,GAAACCGGTAACATCACCAATCTGTGCCCCTTCGGCGAGGTGTTTA...,23427.9549,ETGNITNLCPFGEVFNATTFPSVYAWERKRISNCVADYSVLYNSTS...,NaN,pCWSec
3,[2A-1c],Tor2_RBD_mC2,GTCGACATTGATTATTGACTAGTTATTAATAGTAATCAATTACGGG...,2021-01-26T00:00:00,[14E-6A],40840.0,MGILPSPGMPALLSLVSLLSVLLMGCVAETGNITNLCPFGEVFNAT...,ATGGGGATCCTTCCCAGCCCTGGGATGCCTGCGCTGCTCTCCCTCG...,U0243GA250-11,ACCGGTAACATCACAAATCTGTGCCCCTTCGGCGAGGTGTTTAATG...,MGILPSPGMPALLSLVSLLSVLLMGCVA,ATGGGGATCCTTCCCAGCCCTGGGATGCCTGCGCTGCTCTCCCTCG...,GAAACCGGTAACATCACAAATCTGTGCCCCTTCGGCGAGGTGTTTA...,23493.1739,ETGNITNLCPFGEVFNATKFPSVYAWERKKISNCVADYSVLYNSTF...,NaN,pCWSec
4,"[1I-6g, 1H-4a]",RsSHC014_RBD_mC2_pCWSec,GTCGACATTGATTATTGACTAGTTATTAATAGTAATCAATTACGGG...,2021-01-26T00:00:00,[14E-7A],39350.0,MGILPSPGMPALLSLVSLLSVLLMGCVAETGNITNLCPFGEVFNAT...,ATGGGGATCCTTCCCAGCCCTGGGATGCCTGCGCTGCTCTCCCTCG...,U0243GA250-14,ACCGGTAACATCACCAATCTGTGCCCCTTCGGCGAGGTGTTTAACG...,MGILPSPGMPALLSLVSLLSVLLMGCVA,ATGGGGATCCTTCCCAGCCCTGGGATGCCTGCGCTGCTCTCCCTCG...,GAAACCGGTAACATCACCAATCTGTGCCCCTTCGGCGAGGTGTTTA...,23158.5289,ETGNITNLCPFGEVFNATTFPSVYAWERKRISNCVADYSVLYNSTS...,NaN,pCWSec


# Write JSON

In [43]:
df.to_json(pathing('../data/GenScript.json', new=True, overwrite=True), indent=4, orient='records')

# Manual Spot Checking - Flag columns with multiple entities